## Set Variables

In [1]:
PROJECT_ID = 'jchavezar-demo'
TRAIN_IMAGE = 'gcr.io/jchavezar-demo/pytorch-custom-random-t:v2'
PREDICTION_IMAGE = 'gcr.io/jchavezar-demo/pytorch-custom-random-p:v2'
STAGING_BUCKET = 'gs://vtx-staging'

# Training Block

In [2]:
## Create Folder Code Files Structure
!rm -fr training
!mkdir training

## Create Training Code [PyTorch]

In [3]:
%%writefile training/train.py
#%%
import pandas as pd
from pytorch_tabular import TabularModel
from pytorch_tabular.models import CategoryEmbeddingModelConfig
from pytorch_tabular.config import DataConfig, OptimizerConfig, TrainerConfig, ExperimentConfig
from pytorch_tabular.models.common.heads import LinearHeadConfig

train = pd.read_csv('gs://vtx-datasets-public/pytorch_tabular/synthetic/train.csv')
test = pd.read_csv('gs://vtx-datasets-public/pytorch_tabular/synthetic/test.csv')
val = pd.read_csv('gs://vtx-datasets-public/pytorch_tabular/synthetic/val.csv')

cat_col_names = [col for col in train.columns if 'cat' in col]
num_col_names = [col for col in train.columns if 'num' in col]

data_config = DataConfig(
    target=['target'], #target should always be a list. Multi-targets are only supported for regression. Multi-Task Classification is not implemented
    continuous_cols=num_col_names,
    categorical_cols=cat_col_names,
)
trainer_config = TrainerConfig(
    auto_lr_find=True, # Runs the LRFinder to automatically derive a learning rate
    batch_size=1024,
    max_epochs=100,
    accelerator="auto", # can be 'cpu','gpu', 'tpu', or 'ipu' 
)
optimizer_config = OptimizerConfig()


head_config = LinearHeadConfig(
    layers="", # No additional layer in head, just a mapping layer to output_dim
    dropout=0.1,
    initialization="kaiming"
).__dict__ # Convert to dict to pass to the model config (OmegaConf doesn't accept objects)

model_config = CategoryEmbeddingModelConfig(
    task="classification",
    layers="32-16", # Number of nodes in each layer
    activation="LeakyReLU", # Activation between each layers
    dropout=0.1,
    initialization="kaiming",
    head = "LinearHead", #Linear Head
    head_config = head_config, # Linear Head Config
    learning_rate = 1e-3
)

tabular_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
)

tabular_model.fit(train=train, validation=val)
tabular_model.save_model('/gcs/vtx-models/pytorch/tabular_random')

Writing training/train.py


### Build Image and Push to GCR

In [4]:
%%writefile training/Dockerfile
FROM pytorch/pytorch:1.12.1-cuda11.3-cudnn8-devel

COPY . .
RUN pip install pytorch_tabular[extra]
RUN pip install gcsfs

ENTRYPOINT ["python", "train.py"]

Writing training/Dockerfile


In [ ]:
!gcloud builds submit -t $TRAIN_IMAGE training/.

Creating temporary tarball archive of 2 file(s) totalling 2.2 KiB before compression.
Uploading tarball of [training/.] to [gs://jchavezar-demo_cloudbuild/source/1679412442.518805-8456c156090949af94eba998f7289159.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/jchavezar-demo/locations/global/builds/fbcbcacb-7afc-4680-9aa6-f657485698a2].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds/fbcbcacb-7afc-4680-9aa6-f657485698a2?project=569083142710 ].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "fbcbcacb-7afc-4680-9aa6-f657485698a2"

FETCHSOURCE
Fetching storage object: gs://jchavezar-demo_cloudbuild/source/1679412442.518805-8456c156090949af94eba998f7289159.tgz#1679412442723328
Copying gs://jchavezar-demo_cloudbuild/source/1679412442.518805-8456c156090949af94eba998f7289159.tgz#1679412442723328...
/ [1 files][  1.2 KiB/  1.2 KiB]                                                
Operation completed over 1 

## Run Training CustomJob using Container Image

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, staging_bucket=STAGING_BUCKET)

worker_pool_specs = [
        {
            "machine_spec": {
                "machine_type": "n1-standard-4",
                "accelerator_type": "NVIDIA_TESLA_T4",
                "accelerator_count": 1,
            },
            "replica_count": 1,
            "container_spec": {
                "image_uri": TRAIN_IMAGE,
                "command": [],
                "args": [],
            },
        }
    ]

my_job = aiplatform.CustomJob(
    display_name='pytorch_tabular_custom',
    worker_pool_specs=worker_pool_specs,
)

my_job.run()

# Prediction Block

In [ ]:
## Create Folder Code Files Structure
!rm -fr prediction
!mkdir prediction
!mkdir prediction/app
!mkdir prediction/app/tabular_random

## Create Prediction Code [Uvicorn:FastAPI PyTorch]

In [ ]:
%%writefile prediction/app/main.py
import json
import os
import pandas as pd
from fastapi import Request, FastAPI
from pytorch_tabular import TabularModel

app = FastAPI()
columns = pd.read_csv('gs://vtx-datasets-public/pytorch_tabular/synthetic/train.csv', nrows=0).iloc[:,:-1].columns.to_list()
loaded_model = TabularModel.load_from_checkpoint("../../training/tabular_random_model")

@app.get('/health_check')
def health():
    return 200
if os.environ.get('AIP_PREDICT_ROUTE') is not None:
    method = os.environ['AIP_PREDICT_ROUTE']
else:
    method = '/predict'

@app.post(method)
async def predict(request: Request):
    print("----------------- PREDICTING -----------------")
    body = await request.json()
    instances = body["instances"]
    data_pred = pd.DataFrame([instances],columns=columns)
    print(data_pred)
    outputs = loaded_model.predict(data_pred)
    response = outputs['prediction'].tolist()
    print("----------------- OUTPUTS -----------------")
    return {"predictions": response}

### Build Image and Push to GCR

In [ ]:
%%writefile prediction/Dockerfile
FROM pytorch/pytorch:1.12.1-cuda11.3-cudnn8-devel

COPY app /app
WORKDIR /app

RUN pip install pytorch_tabular[extra]
RUN pip install uvicorn fastapi
RUN pip install gcsfs

CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "8080"]

## Copy Model from GCS

In [ ]:
!gsutil cp -r gs://vtx-models/pytorch/tabular_random prediction/app/tabular_random

## Create Container Image and Push it

In [ ]:
!gcloud builds submit -t $PREDICTION_IMAGE prediction/.

## Upload to Model Registry

In [ ]:
model = aiplatform.Model.upload(
    display_name="synthetic_data_pytorch",
    serving_container_image_uri=PREDICTION_IMAGE,
    serving_container_health_route="/health_check",
    serving_container_ports=[8080]
)

In [ ]:
endpoint = model.deploy(
    deployed_model_display_name='synthetic_data_pytorch',
    machine_type='a2-highgpu-1g',
    accelerator_type='NVIDIA_TESLA_A100',
    accelerator_count=1,
    min_replica_count=1,
    max_replica_count=1
)